In [42]:
from bs4 import BeautifulSoup 
import requests
import math
import re
import pandas as pd
import time
import datetime
import urllib
import os
# import pattern.en as pe
# from PyDictionary import PyDictionary
# from nltk.corpus import wordnet as wn
# import string
import json
import pgeocode
from geopy.geocoders import Nominatim

In [2]:
href_list=[]
for page in range(1,63):
    print(page)
    url=f'https://www.homeshelf.com.au/builders?sortBy=1&page={page}'
    req=requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    for i in soup.select('.linkDetails'):
        url=f'https://www.homeshelf.com.au{i.get("href")}'
        href_list.append(url)
        #print(url)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


In [3]:
print(len(href_list ))

1302


# #Functions

In [4]:
# url=href_list[0]
# # #url='https://www.homeshelf.com.au/builders/new-home-builder-d-construction-nsw-2026/795?trackLink=buildersResults18'
# req=requests.get(url)
# soup = BeautifulSoup(req.text, "html.parser")

# #address=func_address(soup)
# # address='U12/13 Karp Court Bundall Qld 4217,Gold_Coast'
# # geolocator = Nominatim(user_agent="jawad1212")
# # location = geolocator.geocode(address, addressdetails=True)
# # print(location.raw)



# #print(func_suburb(address))
# # print('\n')

# # # a['place_name']
# # # for i in soup.find('span',{'itemprop':'address'}):
# # #     a=i.text
# # #     print((i.text).strip())
# # #     break
    
# # # #description=soup.select('.builder-description')[0].text
# # # <a target="_blank" jstcache="52" href="https://maps.google.com/maps?ll=-33.753372,150.923023&amp;z=13&amp;t=m&amp;hl=en-US&amp;gl=US&amp;mapclient=embed&amp;daddr=50%20Frederick%20St%20Blacktown%20NSW%202148%20Australia@-33.7533722,150.9230228" class="navigate-link"> <div class="icon navigate-icon"></div> <div jstcache="53" class="navigate-text">Directions</div> </a>
# # # description
# # #('.phone')[0]

In [5]:
def func_state_code(zip_code):
    state_code=''
    try:
        nomi = pgeocode.Nominatim('au')
        a=nomi.query_postal_code(zip_code)
        state_code=a['state_code']
    except:
        pass
    return state_code

In [6]:
def func_suburb(address):
    suburb='other'
    try:
        geolocator = Nominatim(user_agent="jawad1212")
        location = geolocator.geocode(address, addressdetails=True)
        suburb=location.raw['address']['suburb']
    except:
        suburb='other'
    return suburb.title()

In [7]:
def func_raw_address(soup):
    address=''
    try:
        address=soup.find('span',{'itemprop':'address'}).text
        address=re.sub('\s+', ' ', address).strip()    
        
    except:
        address=''
    return address

In [46]:
def func_address(raw_address,suburb_list):
    address=''
    
    suburb=func_suburb(raw_address)
    zip_code=func_zip_code(raw_address)
    city=func_city(suburb, suburb_list,zip_code)
    state_code=func_state_code(zip_code)
    
    suburb=suburb.lower()
    city=city.lower()
    try:
        state_code=state_code.lower()
    except AttributeError:
        state_code='NULL'
    raw_address=raw_address.lower()
    
    address=raw_address.split(state_code)[0]
    #print("city : "+city)
    
    if city!='':
            address=address+" ,"+str(city) 
    
    return address.title()
    

In [17]:
def func_zip_code(address):
    zip_code=''
    try:
        regex=re.compile(r'(\d{4})(?=\s|$)')
        zip_code=re.search(regex, address).group()
        
    except:
        zip_code=''
    return zip_code

In [18]:
def func_cell_number(soup):
    cell=''
    index=0
    try:
        cell=soup.find_all('li',{'itemprop':'phone'})[0].text
        cell=(re.sub("[^0-9^]", "",cell))
        if cell.startswith('0'):
            cell="61"+cell[1:]
        elif cell.startswith('1'):
            cell=f"6{cell}"
        else:
            pass
        
        
        if not (len(cell)>=10  and len(cell)<=12):
            cell=''
        
        
    except:
        cell=''
    return cell


In [19]:
def func_company_name(soup):
    company_name=''
    try:
        company_name=soup.select('h1')[0].text
    except:
        company_name=''
    return company_name

In [20]:
def func_description(soup):
    description=''
    try:
        description=soup.select('.builder-description')[0].text
    except:
        description=''
    return description


In [21]:
def empty():
    return ''

In [22]:
def func_user_type():
    return 'Builder'

In [23]:
# for url in href_list[:20]:
#     req=requests.get(url)
#     soup = BeautifulSoup(req.text, "html.parser")
#     raw_address=func_raw_address(soup)
#     zip_code=func_zip_code(raw_address)
#     suburb=func_suburb(raw_address)
#     print(suburb,end=' : ')
#     print(func_city(suburb,city_suburb,zip_code))
#     #address=func_address(soup)
#     #description=func_description(soup)

#     #print(func_state_code(zip_code))
#     #print(func_address(raw_address,city_suburb))
    
# #     zip_code=func_zip_code(address)
# #     print(func_address(soup)+" : "+str(func_city(zip_code)))

In [24]:
def func_created_at():
    return datetime.datetime.now().strftime('%d/%m/%y %H:%M:%S')

In [25]:
def func_bussiness_expertise(description):

    expertise={
    'Residential':['home','plot','house','residential','appartment','cottage','duplex','multi-family'],
    'Commercial':['shop','office','warehouse']}
    bussiness_expertise=[]
    for key in expertise:
        for word in expertise[key]:
            if word in description.lower():
                bussiness_expertise.append(key)
                break
    if bussiness_expertise==[]:
        bussiness_expertise=['Residential','Commercial']
    return bussiness_expertise
    

In [26]:
def func_area(address):
    area=''
    try:
        geolocator = Nominatim(user_agent="jawad1212")
        #address='119 Kampung Baru Balakong'
        location = geolocator.geocode(address, addressdetails=True)
        area=location.raw['address']['municipality']
        
    except:
        pass
    return area

In [27]:
def func_state(zip_code):
    state=''
    try:
        nomi = pgeocode.Nominatim('au')
        a=nomi.query_postal_code(zip_code)
        state=a['state_name']
    except:
        state=''
    return state

In [28]:
def func_city(suburb,suburb_list,zip_code):
    city=''

    if suburb.lower()!='other':
        #try:
        suburb=suburb.lower()
        for key in city_suburb.keys():
            if suburb in city_suburb[key]:
                city=city.replace('_',' ')
                city=key.title()
#         except:
#             city=''
    else:
        try:
            nomi = pgeocode.Nominatim('au')
            a=nomi.query_postal_code(zip_code)
            city=(a['county_name']).title()
        except:
            city=''
        
    return city

In [29]:
def return_austrailia_city():
    df=pd.read_excel(r"C:\Users\Dell\jupyter files\Autrailia\australia_city.xlsx")
    city_list=df['city'].to_list()
    return city_list

In [30]:
def save_user_logo(idd,main_folder,soup):
    try:
        images_o=soup.select('.pad .logo img')[0].get('src')
        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(images_o,main_folder+"upload/"+str(idd)+"/logo.jpg")
        seller_pic="logo"
    except:
        seller_pic=""
    return seller_pic

In [31]:
def save_images(idd,main_folder,soup):
    #idd=1
    try:
        os.mkdir(main_folder+"upload/"+str(idd))
    except FileExistsError:
        pass
    #.image-gallery-picture
    front_image=""
    additional_images=[]
    images_o=''
    img_count=1
    for i in soup.select('#slick_1_0 img'):
        if img_count==1:

            images_o=i.get('src')
            opener=urllib.request.build_opener()
            opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(images_o,main_folder+"upload/"+str(idd)+"/front_image.jpg")
            front_image="front_image"
    #             except:
    #                 continue
        elif img_count<=6:
    #             try:
            images_o=i.get('src')

            opener=urllib.request.build_opener()
            opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(images_o,main_folder+"upload/"+str(idd)+f"/image_{img_count}.jpg")
            additional_images.append(f"image_{img_count}.jpg")
    #             except:
    #                 continu
        else:
            continue

        img_count+=1

    return front_image,additional_images

In [32]:
# save_images(2,r"C:\Users\Dell\jupyter files\Autrailia/",soup)

# reading json

In [202]:
###extarcting main cities with suburbs

In [35]:
with open(r"C:\Users\Administrator\jawad\austrailia\austrailia_suburb_to_city.json", 'rb') as fp:
    city_suburb = json.load(fp)

In [36]:
for key in city_suburb:
    city_suburb[key]=[x.lower() for x in city_suburb[key]]

In [38]:
#city_suburb

# Main Script:

In [53]:
austrailia_builders={}
id=1
main_folder=''

for url in href_list[:]:
    try:
        raw_list=[]

        req=requests.get(url)
        soup = BeautifulSoup(req.text, "html.parser")

        front_image,additional_images=save_images(id,main_folder,soup)
        raw_address=func_raw_address(soup)

        suburb=func_suburb(raw_address)
        zip_code=func_zip_code(raw_address)
        description=func_description(soup)

        print(id)

        raw_list.append(func_company_name(soup))#user_first_name
        raw_list.append(empty())#user_surname
        raw_list.append(empty())#email
        raw_list.append(func_cell_number(soup))#cell_number
        raw_list.append(front_image)#profile_image
        raw_list.append(func_user_type())#user_type
        raw_list.append(func_company_name(soup))#company_name
        raw_list.append(func_cell_number(soup))#cell_1
        raw_list.append(empty())#cell_2
        raw_list.append(empty())#contact_person
        raw_list.append(empty())#phone_1
        raw_list.append(empty())#phone_2
        raw_list.append(func_address(raw_address,city_suburb))#address
        raw_list.append(func_city(suburb,city_suburb,zip_code))#city
        raw_list.append(suburb)#suburb
        raw_list.append(func_state(zip_code))#state
        raw_list.append(zip_code)#zipcode
        raw_list.append(func_area(raw_address))#area
        raw_list.append(empty())#company_email
        raw_list.append(save_user_logo(id,main_folder,soup))#logo
        raw_list.append(func_description(soup))#description
        raw_list.append(empty())#no_year_business
        raw_list.append(empty())#no_agents_office
        raw_list.append(func_created_at())#created_at
        raw_list.append(func_bussiness_expertise(description))#business_expertises
        raw_list.append(url)#link


        austrailia_builders[id]=raw_list
        id+=1
    except:
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277



# saving to XLSX

In [55]:
df=pd.DataFrame.from_dict(austrailia_builders,orient='index')
df.insert(loc=0,column='0',value=df.index)
df.columns=['id','user_first_name','user_surname','email','cell_number','profile_image','user_type','company_name','cell_1','cell_2','contact_person','phone_1','phone_2','address','city','suburb','state','zip_code','area','company_email','logo','description','no_year_business','no_agents_office','created_at','business_expertises','link']
#df.columns=['olx_id','title','amenities','property_type','area_unit','land_area','user_cell','property_purpose','price','description','description_html','front_image','other_images','city','address','created_at','username','user_created_ad',"user_logo",'link']
df.head(3)

id                          user_first_name user_surname email  \
1   1                        Woodsman Projects                      
2   2                          Metronest Homes                      
3   3  Stroud Homes Melbourne Outer North East                      

   cell_number profile_image user_type  \
1  61300268910   front_image   Builder   
2  61300510733   front_image   Builder   
3                front_image   Builder   

                              company_name       cell_1 cell_2  ... zip_code  \
1                        Woodsman Projects  61300268910         ...     3082   
2                          Metronest Homes  61300510733         ...     3029   
3  Stroud Homes Melbourne Outer North East                      ...     3081   

                 area company_email  logo  \
1  City of Whittlesea                logo   
2     City of Wyndham                logo   
3     City of Banyule                logo   

                                         description no_year_business  \
1  \nWoodsman Projects is a young, dynamic buildi...                    
2  \nFor us at Metronest Homes all our customers ...                    
3  \nStroud Homes Melbourne Outer North East Buil...                    

  no_agents_office         created_at        business_expertises  \
1                   15/12/20 15:29:35  [Residential, Commercial]   
2                   15/12/20 15:29:37              [Residential]   
3                   15/12/20 15:29:38              [Residential]   

                                                link  
1  https://www.homeshelf.com.au/builders/new-home...  
2  https://www.homeshelf.com.au/builders/new-home...  
3  https://www.homeshelf.com.au/builders/new-home...  

[3 rows x 27 columns]

In [58]:
#df.drop_duplicates(subset='link')

In [59]:
df.to_excel('austrailia_builders_1300.xlsx',index=False)